In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
sys.path.append("/home/ray__/ssd/BERT/")
sys.path.append("/home/ray__/CS/org/etherlabs/ai-engine/pkg/")
from gpt_feat_utils import GPT_Inference

gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/se/epoch3/", device="cuda")
#gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/ether_v2/ether_googleJan13_groupsplit_withstop_4+w_gt3s_lr3e-5/",device="cuda")
# gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/product/", device="cuda")

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [38]:
import json,pickle 
# S.E --current
ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/se/se_entity_feats_v3.pkl","rb"))
sent_dict = pickle.load(open("/home/ray__/ssd/minds/se/se_sent_dict_v3.pkl", "rb"))
# ent_graph = nx.read_gpickle("/home/ray__/ssd/minds/se/se_ent_graph_wcosine_pruned_v3.gpkl")
# label_dict = pickle.load(open("/home/ray__/ssd/minds/ether/ether_single_label_dict_v1.pkl","rb"))
com_map = pickle.load(open("/home/ray__/ssd/minds/se/com_map.pkl", "rb"))
ranked_com = pickle.load(open("/home/ray__/ssd/minds/se/ranked_com.pkl", "rb"))
noun_graph = pickle.load(open("/home/ray__/ssd/minds/se/se_noun_graph.pkl", "rb"))
kp_entity_graph = pickle.load(open("/home/ray__/ssd/minds/se/deployment/kp_entity_graph.pkl", "rb"))

common_entities = ent_fv_full.keys() & com_map.keys()
ent_fv = {}
for ent in common_entities:
    #if ent in sent_dict.keys(): #and label_dict[ent]=='PER':
    #    continue
    #if len(sent_dict[ent])>2:
    ent_fv[ent] = ent_fv_full[ent]

In [9]:
import sys
sys.path.append("../")
from filter_groups import CandidateKPExtractor

kp_e = CandidateKPExtractor()
uncased_nodes = [ele.lower() for ele in kp_entity_graph]
uncased_node_dict = dict(zip(list(kp_entity_graph),uncased_nodes))

[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /tmp/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [57]:
text = "What this is asking for is a character to append to this unique user ID right here so that I can find it in the job. So let me Cru let me just say I want to do you excuse me a wife or YouTube. It says it's submitted. Now what I'm going to be able to see when I hit enter again are the results of my job run and I see I have a A return code of eight so let's go and find that so let me hit enter again and there are two ways that I can go and find the particular output and I got to get to S DS F anyway, so what I can do is hit F3 several times till I get to the main primary option menu and SDS F and S case is option 13 so I can go to the screen and hit 13. Or let me go back to where I was at when my JCL and I can use what's called a fast path. And if I know the option is 13, I can hit equals 13. It takes me to the same spot now. I need to hit o to look at my output Q naught 0. Oh and now I need to do a find on my particular job. And that was a y and I see two outputs and let me see. What's going on? With my COBOL. I'm just going to hit select by the COBOL in order to scroll down. I need to hit f8.  if you will think I did that on purpose when I really did not if you look at the X on this it's actually Midnight when I'm doing this, so let me if three out of here to save that. Go back to my JCL."

In [58]:
import networkx as nx
import json
import numpy as np
from scipy.spatial.distance import cosine
import statistics
import text_preprocessing.preprocess as tp
import pickle

In [59]:
from extra_preprocess import preprocess_text
sent_list = preprocess_text(text)
sent_fv = [gpt_model.get_text_feats(sent) for sent in sent_list]

In [60]:
G = nx.Graph()
G.add_nodes_from(range(len(sent_fv)))

In [61]:
import statistics

node_list = range(len(sent_fv))
for index1, nodea in enumerate(range(len(sent_fv))):
    for index2, nodeb in enumerate(range(len(sent_fv))):
        if index2 >= index1:
            c_score = 1 - cosine(sent_fv[nodea], sent_fv[nodeb])
            #if c_score>= outlier_score:
            G.add_edge(nodea, nodeb, weight = c_score)
    closest_connection_n = sorted(dict(G[nodea]).items(), key=lambda kv:kv[1]["weight"], reverse=True)
    weights_n = list(map(lambda kv: (kv[1]["weight"]).tolist(), closest_connection_n))
    q3 = np.percentile(weights_n, 75)
    iqr = np.subtract(*np.percentile(weights_n, [75, 25]))
    #outlier_score = q3 + (1.5 * iqr)
    outlier_score = q3 + (1 * iqr)
    for nodeb, param in dict(G[nodea]).items():
        if param['weight']>=q3:
            pass
        else:
            G.remove_edge(nodea, nodeb)
        
    print ("Updated node: ", index1)

Updated node:  0
Updated node:  1
Updated node:  2
Updated node:  3
Updated node:  4
Updated node:  5
Updated node:  6
Updated node:  7
Updated node:  8
Updated node:  9


In [62]:
from community import best_partition

comm_temp = best_partition(G, resolution=1)

In [63]:
prev = 0 
com_map = {}
for ent, cls in sorted(comm_temp.items(),key=lambda kv:kv[1]):
    if prev!=cls:
        print ("----------------------Cluster-----------------")
        prev = cls
    
    print (sent_list[ent],"\n")
    if cls in com_map.keys():
        com_map[cls].append(ent)
    else:
        com_map[cls] = [ent]

What this is asking for is a character to append to this unique user ID right here so that I can find it in the job. 

Oh and now I need to do a find on my particular job. 

----------------------Cluster-----------------
So let me Cru let me just say I want to do you excuse me a wife or YouTube. 

----------------------Cluster-----------------
Now what I am going to be able to see when I hit enter again are the results of my job run and I see I have a A return code of eight so let us go and find that so let me hit enter again and there are two ways that I can go and find the particular output and I got to get to S DS F anyway, so what I can do is hit FXnumberX several times till I get to the main primary option menu and SDS F and S case is option XnumberX so I can go to the screen and hit XnumberX. 

And if I know the option is XnumberX, I can hit equals XnumberX. 

----------------------Cluster-----------------
Or let me go back to where I was at when my JCL and I can use what is call

In [64]:
com_map

{0: [0, 6], 1: [1], 2: [2, 4], 3: [3], 4: [5], 5: [7], 6: [8], 7: [9]}

In [67]:
agg_fv = {}
for comm, s_list in com_map.items():
    if len(s_list)>1:
        temp_fv = [sent_fv[s] for s in s_list]
        agg_fv[comm] = np.mean(temp_fv, axis=0)

In [68]:
dist_list = {}
for pos, fv in agg_fv.items():
    temp_list = []
    for entity in ent_fv.keys():
        temp_list.append((entity, 1-cosine(ent_fv[entity], fv)))
    dist_list[pos] = sorted(temp_list, key=lambda kv:kv[1], reverse=True)[:10]

In [69]:
for pos_sent, similar_entities in dist_list.items():
    print ("Sent: \n", [sent_list[s] for s in  com_map[pos_sent]])
    print ("\n Similar Entities", list(map(lambda kv: kv[0], similar_entities)))
    print ("\n\n")

Sent: 
 ['What this is asking for is a character to append to this unique user ID right here so that I can find it in the job.', 'Oh and now I need to do a find on my particular job.']

 Similar Entities ['Id', 'Ok', 'Choose', 'Fyi', 'Name', 'Uid', 'Find', 'Explain', 'User', 'Title']



Sent: 
 ['Now what I am going to be able to see when I hit enter again are the results of my job run and I see I have a A return code of eight so let us go and find that so let me hit enter again and there are two ways that I can go and find the particular output and I got to get to S DS F anyway, so what I can do is hit FXnumberX several times till I get to the main primary option menu and SDS F and S case is option XnumberX so I can go to the screen and hit XnumberX.', 'And if I know the option is XnumberX, I can hit equals XnumberX.']

 Similar Entities ['Typo', 'Z', 'Y', 'Jnz', 'Mychar', 'X', 'Strncmp', 'Nerd', 'Xor', 'Operator']





# dummy

In [1]:
import logging
import nltk
from nltk.corpus import stopwords
import pickle

logger = logging.getLogger(__name__)


stop_words = set(stopwords.words("english"))
stop_words_spacy = list(
    """
a about above across after afterwards again against all almost alone along
already also although always am among amongst amount an and another any anyhow
anyone anything anyway anywhere are around as at

back be became because become becomes becoming been before beforehand behind
being below beside besides between beyond both bottom but by

call can cannot ca could

did do does doing done down due during

each eight either eleven else elsewhere empty enough even ever every
everyone everything everywhere except

few fifteen fifty first five for former formerly forty four from front full
further

get give go

had has have he hence her here hereafter hereby herein hereupon hers herself
him himself his how however hundred

i if in indeed into is it its itself

keep

last latter latterly least less

just

made make many may me meanwhile might mine more moreover most mostly move much
must my myself

name namely neither never nevertheless next nine no nobody none noone nor not
nothing now nowhere n't

of off often on once one only onto or other others otherwise our ours ourselves
out over own

part per perhaps please put

quite

rather re really regarding

same say see seem seemed seeming seems serious several she should show side
since six sixty so some somehow someone something sometime sometimes somewhere
still such

take ten than that the their them themselves then thence there thereafter
thereby therefore therein thereupon these they third this those though three
through throughout thru thus to together too top toward towards twelve twenty
two

under until up unless upon us used using

various very via was we well were what whatever when whence whenever where
whereafter whereas whereby wherein whereupon wherever whether which while
whither who whoever whole whom whose why will with within without would

yet you your yours yourself yourselves yeah okay
""".split()
)
stop_words = set(list(stop_words) + list(stop_words_spacy))

kp_entity_graph = pickle.load(open("/home/ray__/ssd/minds/se/se_ent_kp_graph_fromsd.pkl", "rb"))

class GraphFilter(object):
    def __init__(self, s3_client=None, graph_file_path=None):
        self.kp_graph = kp_entity_graph
        # with open("entity_noun_graph.pkl", "rb") as f_:
        #     self.kp_graph = pickle.load(f_)

        logger.info("Downloaded and loaded ENT-KP graph")

    def get_segment_nouns(self, segment_text_list):
        segment_noun_list = []
        for text in list(segment_text_list):
            tagged_sents = nltk.pos_tag_sents(
                nltk.word_tokenize(text) for sent in nltk.sent_tokenize(text)
            )
            text_kps = []
            for tagged_sent in tagged_sents:
                text_kps.extend(
                    [ele[0] for ele in list(tagged_sents[0]) if ele[1].startswith("NN")]
                )
            segment_noun_list.append(text_kps)

        return segment_noun_list

    def filter_keyphrases(self, phrase_dict, segment_text_list):
        filtered_kp_list = []
        dropped_kp_list = []

        segment_noun_list = self.get_segment_nouns(segment_text_list=segment_text_list)

        for text, nouns in zip(segment_text_list, segment_noun_list):
            sent_nouns = [ele.lower() for ele in nouns]

            for kps, scores in phrase_dict.items():
                kp_nouns = self.get_kp_nouns(kps, sent_nouns)
                if set(kp_nouns) & set(self.kp_graph) == set(kp_nouns):
                    filtered_kp_list.append({kps: scores})
                else:
                    dropped_kp_list.append({kps: scores})

        return filtered_kp_list, dropped_kp_list

    def get_kp_nouns(self, kp, sent_nouns):
        return set(kp.lower().split(" ")) & set(sent_nouns)

    def filter_entities(self, phrase_dict, segment_text_list):
        filtered_ent_list = []
        dropped_ent_list = []

        # segment_noun_list = self.get_segment_nouns(segment_text_list=segment_text_list)

        for text in segment_text_list:
            for ents, scores in phrase_dict.items():
                ent = self.get_ent(ents)
                if set(ent) & set(self.kp_graph) == set(ent):
                    filtered_ent_list.append({ents: scores})
                else:
                    dropped_ent_list.append({ents: scores})

        return filtered_ent_list, dropped_ent_list

    def get_ent(self, ent):
        return set(ent.lower().split(" "))

In [2]:
kobj = GraphFilter()

In [3]:
import sys
sys.path.append("/home/ray__/CS/org/etherlabs/ai-engine/pkg/")
import text_preprocessing.preprocess as tp

[nltk_data] Downloading package wordnet to /tmp/nltk_data...
[nltk_data] Error downloading 'wordnet' from
[nltk_data]     <https://raw.githubusercontent.com/nltk/nltk_data/gh-
[nltk_data]     pages/packages/corpora/wordnet.zip>:   [Errno 104]
[nltk_data]     Connection reset by peer
[nltk_data] Downloading package stopwords to /tmp/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /tmp/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [14]:
segment_text = "Yes, I so come back and we were already settling in vital that you can so the design which is report from slacks slack annoyed. But I asked you didn't like the designs from Guam summary and in Asia send some crazy designs, but once we are done with other things and then you can pick up. Okay."
segment_text_list = tp.preprocess(segment_text)

In [15]:
noun_list = kobj.get_segment_nouns(segment_text_list)
phrase_list = {}

In [16]:
phrase_dict = {kp: 0 for kps in noun_list for kp in kps}

In [17]:
kobj.filter_keyphrases(phrase_dict, segment_text_list)

([{'design': 0},
  {'report': 0},
  {'designs': 0},
  {'Guam': 0},
  {'Asia': 0},
  {'send': 0},
  {'crazy': 0},
  {'things': 0},
  {'Okay': 0},
  {'design': 0},
  {'report': 0},
  {'designs': 0},
  {'Asia': 0},
  {'send': 0},
  {'Okay': 0},
  {'design': 0},
  {'report': 0},
  {'designs': 0},
  {'Guam': 0},
  {'Asia': 0},
  {'send': 0},
  {'crazy': 0},
  {'things': 0}],
 [{'Guam': 0}, {'crazy': 0}, {'things': 0}, {'Okay': 0}])

In [18]:
kobj.filter_entities(phrase_dict, segment_text_list)

([{'design': 0},
  {'report': 0},
  {'designs': 0},
  {'Asia': 0},
  {'send': 0},
  {'design': 0},
  {'report': 0},
  {'designs': 0},
  {'Asia': 0},
  {'send': 0},
  {'design': 0},
  {'report': 0},
  {'designs': 0},
  {'Asia': 0},
  {'send': 0}],
 [{'Guam': 0},
  {'crazy': 0},
  {'things': 0},
  {'Okay': 0},
  {'Guam': 0},
  {'crazy': 0},
  {'things': 0},
  {'Okay': 0},
  {'Guam': 0},
  {'crazy': 0},
  {'things': 0},
  {'Okay': 0}])

In [31]:
for nodes in kp_entity_graph:
    if kp_entity_graph.nodes[nodes]['sent_ctr'] < 20:
        print (nodes, "  : ", kp_entity_graph.nodes[nodes]['sent_ctr'])

Orderer   :  10
Channelall   :  18
Sacc   :  4
Sacc Chaincode   :  2
Channel12   :  16
Byfn   :  11
Genesis Block   :  4
Org1   :  17
Org2   :  16
Docker Command   :  7
Node 1   :  4
Bridge   :  9
Fabric   :  9
Gitlab Runner   :  14
Lambda Layers   :  5
Terragrunt   :  11
Gitlab-runner   :  5
Ec2 Spot   :  2
Executor   :  18
Shell Executor   :  2
Golang Lambda   :  2
Amazon Linux Ami   :  3
Img2lambda   :  2
Lambda Layer   :  2
Voila   :  10
Concept   :  8
Pubg   :  2
Blackbox   :  5
Icmp   :  14
Cloudprober   :  16
Prober   :  3
Pingdom   :  11
Ping   :  16
Cloud Platform   :  12
Prometheus Server   :  6
Create React   :  6
Ec2 Service   :  2
React Cluster   :  3
Configure   :  16
Enter React   :  2
Elastic Container Registry   :  5
Ecs Service   :  3
Touch Dockerfile   :  2
Task   :  17
Family   :  3
P3   :  4
Step Functions   :  8
Amazon Dynamodb   :  12
Scikit   :  7
Nlp   :  19
Joblib   :  2
Re   :  14
Invent 2018   :  2
Ml Inference Lambda   :  2
Pipenv   :  6
Lambda Runtime   : 

Vcxproj   :  2
Ant   :  5
Arrays   :  7
Jenkins Script   :  4
Feature Branch   :  14
Kubernetes Clusters   :  7
Kube-opex-analytics   :  2
Koa_db_location   :  2
Angular Devops Series   :  2
Tim Deschryver   :  3
Ct   :  7
Root Directory   :  3
Firebaserc   :  2
Firebase Json   :  6
Firebase App   :  6
Ng   :  7
Circleci Site   :  2
Aws Cloudfront   :  8
Firebase_token   :  2
Intersection Observer   :  4
Bang   :  6
Sitecore   :  2
Hatch   :  4
Marie Kondo   :  6
Routes   :  18
Refactor   :  14
Password   :  13
Gitleaks   :  4
Keys   :  10
-f   :  1
Webdriver   :  16
Selenium Grid   :  15
Selenium Server   :  2
Chromedriver   :  4
Approach   :  5
Alternative   :  2
Boards   :  3
Banking   :  3
Gemba   :  3
Pdca   :  3
Plan   :  10
Act   :  10
Six Sigma   :  2
Smtp   :  12
Notification   :  2
Events   :  14
Nsq   :  2
Financial Times   :  4
Ft   :  10
Project Spawn   :  3
Android Debug Bridge   :  3
Adb   :  14
Usb   :  14
Android Debug   :  2
South Korea   :  6
Netflow   :  2
Isps   : 

Apps   :  17
Ab   :  17
Lineage   :  4
Sa   :  4
Amazon Web   :  12
Palantir   :  3
Dynomite   :  4
Pinners   :  13
Ads   :  17
Pinner   :  6
Ctr   :  7
Account   :  11
Click   :  11
Ads System   :  2
Sql-like   :  12
Downstream   :  8
Liquan Pei   :  2
Bo Liu   :  3
Api 28   :  2
Android 9 0   :  2
Pojos   :  5
Loaders   :  2
Executors   :  4
Coroutines   :  8
Kotlin Coroutines   :  7
Jake Wharton   :  4
Spacex   :  2
Coroutine   :  2
Android Architecture   :  4
Room   :  15
Objectbox   :  3
Viewmodel   :  14
Livedata   :  13
Activity   :  4
Fragment   :  2
Dagger 2   :  5
Koin   :  2
Binder   :  3
Kotlin Extensions   :  2
Eventbus   :  5
Module   :  10
Mode   :  4
Android Pie   :  2
Helper   :  9
Oreo   :  2
Java Devs   :  4
Angellist   :  11
Jakarta Ee   :  2
Eclipse Foundation   :  2
State   :  12
Vert X   :  19
Spring Framework   :  5
Amazon Aws   :  15
Google Spanner   :  8
Newsql   :  2
Jessica Kerr   :  4
Tables   :  3
Source-to-image   :  4
S2i   :  6
Django Migration   :  2
L

Cities   :  2
Et Voila   :  5
Saturn   :  2
Jupiter   :  4
Ssrf   :  10
Ec2 Metadata   :  2
Ec2 Metadata Service   :  4
Metadata Service   :  5
Aws Sdks   :  4
Defense   :  6
Credential   :  5
Identity and Access Management   :  6
Instance-id   :  3
Aws Apis   :  2
Iam Service   :  3
Xen   :  5
Kvm   :  3
Hyper-v   :  9
Platform as a Service   :  4
Google Cloud Services   :  3
Paas Cloud   :  2
Portuguese   :  3
Digital Archive   :  2
Laravel Framework   :  2
Buckets   :  4
Processemailbuckets   :  2
Spatie   :  6
Pointer   :  7
Graalvm Ee   :  6
Clang   :  5
Printf   :  10
Heap   :  12
Libc   :  5
Segfault   :  3
Syscalls   :  2
Java Vm   :  2
E G   :  8
Enterprise Edition   :  4
Mapr   :  2
Towards Data Science   :  3
Hadoop Ecosystem   :  4
Aws Redshift   :  2
Lambda Architecture   :  14
Splice Machine   :  2
Walmart   :  12
John Smith   :  4
Developer Portal   :  7
Jpeg   :  11
Ack   :  7
Ph   :  4
Astronomy   :  4
Zwicky Transient Facility   :  2
Ztf   :  9
Synoptic Survey   :  2


Continuous Integration   :  10
Jenkins Instance   :  3
Github Repos   :  6
Webserver   :  15
Easy Peasy   :  2
Docker-machine   :  2
Production   :  18
Jenkins Pipeline   :  7
Subdomain   :  7
Osquery   :  18
Houston   :  7
Osquery Interactive Shell   :  3
Interactive Shell   :  2
Web01   :  3
Docker_containers   :  3
Docker_images   :  2
Docker_container_processes   :  2
Users   :  6
Groups   :  6
Subquery   :  6
Container Id   :  3
Prune   :  2
Solid Principles   :  11
Single Responsibility Principle   :  14
Liskov Substitution Principle   :  6
Barbara Liskov   :  7
Principle   :  13
Bertrand Meyer   :  4
Subclass   :  5
Lsp   :  6
Interface Segregation Principle   :  4
Dependency Inversion   :  2
Mysqlconnection   :  4
Passwordreminder   :  6
Dbplyr   :  4
Dplyr   :  3
Con   :  2
Rvest   :  2
Xml2   :  2
K-means   :  6
Kml   :  3
Rstudio   :  12
Schemas   :  3
Dataframe   :  19
Gganimate   :  3
Ggplot2   :  4
Mpeg   :  4
Dt   :  14
Rmarkdown   :  3
Codepusher   :  9
Ot   :  12
Share

...

discontinues products   :  1
toxic decision   :  1
odds with joel   :  1
previous excellent   :  1
sane advice on software development   :  1
… developers   :  1
little wasabi universe   :  1
atlassian   :  1
bug-tracking market   :  1
building trello   :  1
creek   :  1
contemporary technologies   :  1
everything from weekly meals   :  1
weddings   :  1
animal shelters   :  1
young bootstrapped startup   :  1
third product   :  1
stable company like fog creek   :  1
ironic twist   :  1
fog creek   :  2
old nemesis   :  1
ceo anil dash   :  1
single-product company   :  1
first product   :  1
applicable way   :  1
excel   :  1
vs code look   :  1
bake   :  3
project configurations   :  1
projects in interactive mode   :  1
include list in bake_config.h   :  1
git url   :  1
sponsored product review   :  1
very first exploration of fpga tech   :  1
interest in iot   :  1
two-dimensional grid of semiconductor modules   :  1
many rows   :  1
dev kit   :  1
such tiny modules   :  1
simple 

viewstate   :  2
small part of popularmoviesviewmodel tests suit   :  1
other git user interface   :  1
commit message template   :  1
harry roberts   :  1
great post   :  1
package.json configuration   :  1
.git/hooks folder   :  1
complete commitizen experience   :  1
repository commitizen   :  1
commit script in package.json   :  1
@ carmenpopoviciu on twitters   :  1
caleb thompson   :  1
message template plugin   :  1
iks cluster context for use   :  1
kube configuration   :  1
clusters via kubectl   :  1
-d option   :  1
cluster configs   :  1
.zshrc file   :  1
cluster context   :  2
kube-ps1 plugin   :  1
add tools   :  1
shell window changes   :  1
other shells   :  1
specific kube config file   :  1
production encoders   :  1
subjective quality   :  1
available encoder tools   :  1
visual appearance   :  1
indicators like speed   :  1
operates   :  1
convex hull   :  1
overall rate-distortion   :  1
optimal compression path   :  1
encode across qualities   :  1
tech blog   : 

internet protocol   :  1
requested information   :  1
right computer   :  1
right application   :  1
conjunction with various other protocols   :  1
facebook post   :  1
introductory technical blog post   :  1
virtual server   :  1
ec2 instances list   :  1
application stack configuration   :  1
inter-component communication   :  1
private ip/dns name   :  1
application services layer   :  1
domain model layer   :  1
template engines   :  1
javascript frameworks   :  1
organized way   :  1
onion architecture   :  2
save changes   :  1
region-wide settings   :  1
standard plan   :  1
terraform documentation   :  1
official api_gateway_method_settings terraform   :  1
fmn   :  2
parsimonious new syntax for if-then-else conditional tests   :  1
uml   :  1
fmn specification   :  1
simple transition   :  1
initial game start state   :  1
wander around base state   :  1
frame machine notation   :  1
such ambiguity   :  1
creation of machine specifications   :  1
topic for fmn specifications 

service inquiries   :  1
whatsapp business profile   :  1
german language   :  1
corresponding bot answers   :  1
pure url   :  1
agent view   :  1
multiple messages   :  1
defined delay   :  1
media like images   :  1
towards serverless architecture   :  1
common spam   :  1
source distribution   :  1
different emails   :  1
target distribution   :  1
word2vec   :  1
vector of coordinates   :  1
x = vec   :  1
'berlin   :  1
— vec   :  1
'germany   :  1
+ vec   :  1
'france   :  1
original blog   :  1
actual build environment   :  1
omniscidb   :  2
tarball downloads   :  1
omnisci cloud   :  1
14-day free trial   :  1
nvidia cuda dependencies   :  1
common dependencies   :  1
gpu support   :  1
gpu-enabled build   :  1
iam role for lambda function   :  1
acme-dns-route53.zip file   :  1
lambda services   :  1
lambda-acme-dns-route53-executor role   :  1
sns publish permission   :  1
new certificate   :  1
letsencrypt_email   :  1
encrypt expiration email   :  1
//~/acme-dns-route53.z

hyperasmediumexample   :  1
own notebook   :  1
common strategies today   :  1
so-called git workflows   :  1
successful testing   :  1
developer branch   :  1
data with charts   :  1
first docker command   :  1
zipkin image from __url__   :  1
next docker command   :  1
`dockerfile` as below   :  1
customer details   :  2
application.properties for spring   :  1
below curl   :  1
click on `create index pattern`   :  1
click on 'next step   :  1
kafka journey   :  1
checkr   :  2
background check   :  1
large state machine   :  1
screenings   :  1
request objects   :  1
consumer group   :  1
publish errors   :  1
consumer errors   :  1
additional tooling   :  1
flagr   :  1
fault-tolerant design decisions   :  1
zhu   :  1
zhuojie zhou   :  1
jason dougherty   :  1
eric psalmond   :  1
data for storage   :  1
gb per month   :  1
standard status   :  1
byte range   :  1
range http request header   :  1
latency penalty   :  1
window queries   :  1
spatio-temporal data   :  1
discrete ser

In [30]:
kp_entity_graph.nodes['Asia']

{'node_type': 'entity',
 'node_label': 'LOC',
 'para_ctr': 13,
 'art_ctr': 13,
 'art_freq_list': [1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1],
 'sent_ctr': 14}

In [33]:
import pandas as pd
meeting_data = pd.read_csv("/home/ray__/ssd/engineering_meetings_goog_only_13Jan.csv")

In [41]:
meeting_list = list(set(meeting_data.sort_values("createdAt", ascending=False)['meeting_id']))[:10]

In [44]:
meeting_data.iloc

Unnamed: 0                                            0
confidence                                            0
createdAt                    2020-01-03 06:59:34:402608
deleted                                           False
deletedAt                                           NaN
duration                                              2
endTime                            2020-01-03T06:59:34Z
filteredText                                        NaN
id                 bb212886-0e46-40f3-9c16-6eb2e115533a
isEndOfSentence                                   False
language                                            NaN
languageCode                                      en-IN
originalText                                        NaN
recordingId        b7f65222-0182-4568-99c2-a0a0a88e0cbd
spokenBy               b1e8787a9a1f4859ac11cbb6a8124fd9
startTime                          2020-01-03T06:59:32Z
status                                 no_transcription
transcriber                           google_spe

In [46]:
def get_grouped_dfs(data_df,num_meetings=10):
    meeting_dfs = []
    meet_ctr = 0
    for id,g in data_df.groupby('meeting_id'):
        meeting_dfs.append(g.sort_values(by="startTime").drop("Unnamed: 0",axis=1))
        meet_ctr+=1
        if meet_ctr==num_meetings:
            break
    return meeting_dfs

ether_meetings = pd.read_csv('/home/ray__/ssd/engineering_meetings_goog_only_13Jan.csv').reset_index(drop=True).dropna(subset=["originalText"]).drop_duplicates(subset=["originalText"]).sort_values(by='startTime',ascending = False).reset_index(drop=True)
master_dfs = get_grouped_dfs(ether_meetings,num_meetings=10)

In [73]:
meeting_list = []
for index, dfs in enumerate(master_dfs):
    meeting = {}
    meeting['contextId'] = "01DBB3SN99AVJ8ZWJDQ57X9TGX"
    meeting['mindId'] = "01DAAYHEKY5F4E02QVRJPTFTXV"
    meeting['instanceId'] = "8225dafd-0ccc-4059-9d53-6996d496901a"
    meeting['segments'] = []
    segobj = {}
    for pos in dfs.iterrows():
        meeting['segments'].append(dict(pos[1]))
    meeting_list.append(meeting)

In [74]:
meeting_list

[{'contextId': '01DBB3SN99AVJ8ZWJDQ57X9TGX',
  'mindId': '01DAAYHEKY5F4E02QVRJPTFTXV',
  'instanceId': '8225dafd-0ccc-4059-9d53-6996d496901a',
  'segments': [{'confidence': 0.83152102,
    'createdAt': '2019-07-24 06:11:54:856927',
    'deleted': False,
    'deletedAt': nan,
    'duration': 12.0,
    'endTime': '2019-07-24T06:11:51Z',
    'filteredText': nan,
    'id': 'e4da2d529a0246008f6c3c487de2ca28',
    'isEndOfSentence': nan,
    'language': nan,
    'languageCode': 'en-US',
    'originalText': "A couple of things that color noticed and is doesn't happen. Staging also so deep you want to talk about that?. ",
    'recordingId': 'a9216bb1f165421f9fdbd1ca1d01f02e',
    'spokenBy': '8d6db5f7d9b74c54ba38fe710ffcaf3f',
    'startTime': '2019-07-24T06:11:39Z',
    'status': 'completed',
    'transcriber': 'google_speech_api',
    'transcriptId': '0efc2729-d93a-42f8-82c2-848047b78d7f',
    'updatedAt': '2019-07-24T06:12:16.212312234Z',
    'meeting_id': '0366634106bd40ffb76b07e8a03d0c71'

In [75]:
pickle.dump(meeting_list, open("last_10_meeting_dump.json","wb"))

In [76]:
def get_grouped_dfs(data_df,num_meetings=10):
    meet_ctr = 0
    meeting_dfs = data_df.groupby(['meeting_id']).agg({'startTime':"first","originalText":list}).sort_values(by="startTime",ascending=False).reset_index()[:num_meetings]
    return meeting_dfs

In [77]:
ether_meetings = pd.read_csv('/home/ray__/ssd/engineering_meetings_goog_only_13Jan.csv').reset_index(drop=True).dropna(subset=["originalText"]).drop_duplicates(subset=["originalText"]).sort_values(by='startTime',ascending = False).reset_index(drop=True)
master_dfs = get_grouped_dfs(ether_meetings,num_meetings=10)

,meeting_id,startTime,originalText
0,fb61b52a1ca547a3a22c55c8804e9090,2020-01-03T06:56:24Z,"[Anything else you want to chat? , So like the..."
1,1d6f6c2f8dcf404197dacd0f861e7445,2020-01-01T16:22:08Z,[And then the other one that lay down into wha...
2,41af7064cbd04e30a6f2c2fcdb73db73,2019-12-30T13:31:43Z,"[Okay. Thank you. , Two seconds and resume is ..."
3,bb55fee331914122ae1e1fc7d0325372,2019-12-23T06:01:05Z,"[Thank you. , rest , Push the men become the s..."
4,683270b99f424e3db8b7c721685fb085,2019-12-19T10:44:48Z,[Okay. I like coffee. So I guess actions are w...
5,2112d8d50bd649aeb5424e5c22ea0244,2019-12-19T06:26:29Z,"[Thanks guys. , Alright anything else to talk ..."
6,6be04a915e46490f8d8ecf3620f5333c,2019-12-17T06:39:57Z,[I'm working on supporting wrong media files f...
7,333f727d1773447d9e6f5280cbcc5f73,2019-12-14T10:54:06Z,[That's just a small experiment which I'm just...
8,498d08b439304f9f9b8988558656b55a,2019-12-13T12:22:54Z,[there's some also going to make a quick imple...
9,917dc5ce296f4e5ca41f23003646f1a1,2019-12-13T10:31:36Z,[I'm good. Can you upload the transcripts for ...


# entity Strength

In [86]:
ent_list = [('Action Mailbox', 0.4386250376701355), ('Actions Api', 0.4351121485233307), ('Trigger', 0.43024733662605286), ('Store Js', 0.42989540100097656), ('Post Method', 0.4287060499191284), ('File_name', 0.42814451456069946), ('Events Json', 0.42587754130363464), ('Leaderboard', 0.42543911933898926), ('States', 0.4251515567302704), ('Events', 0.4233638048171997), ('Old', 0.4464528560638428), ('Tiger', 0.44128546118736267), ('Aim', 0.4400998055934906), ('Basecamp 2', 0.438770055770874), ('Frontend Javascript', 0.43804675340652466), ('Chime', 0.437365859746933), ('Shep', 0.4368863105773926), ('Ipad', 0.4364376962184906), ('Yew', 0.4353457987308502), ('Basecamp', 0.4335535764694214)]
ent_list = list(map(lambda kv: kv[0], ent_list))

In [87]:
sub_graph = kp_entity_graph.subgraph(ent_list)

In [89]:
sub_graph.number_of_edges()

2